In [62]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import clean_df

load_dotenv(dotenv_path="../.env")
file_path = os.getenv("FILEPATH2")

df = pd.read_csv(file_path)

In [63]:
df_cleaned = clean_df(df)

# Creating Month column 
df_cleaned['Transaction Date'] = pd.to_datetime(df_cleaned['Transaction Date'])
df_cleaned['Month'] = df_cleaned['Transaction Date'].dt.to_period('M')

df_cleaned.head(3)

c:\Users\matt\Desktop\DataProjects\ad_hoc\finances_streamlit_tracker\utils.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")
c:\Users\matt\Desktop\DataProjects\ad_hoc\finances_streamlit_tracker\utils.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount'] = df['Amount'].abs()
c:\Users\matt\Desktop\DataProjects\ad_hoc\finances_streamlit_tracker\utils.py:10: SettingWithCopyWarning: 
A value is trying to be set on a cop

,Transaction Date,Post Date,Description,Category,Type,Amount,Was Venmoed,Split Count,Month
0,2025-10-29,10/30/2025,FSP*ROYAL OAK GOLF CEN,Health & Wellness,Sale,14.50,,1,2025-10
1,2025-10-21,10/22/2025,DTE ENERGY,Bills & Utilities,Sale,113.46,,1,2025-10
5,2025-10-05,10/6/2025,Spotify USA,Bills & Utilities,Sale,11.99,,1,2025-10


In [ ]:
# 1) Recurring subscriptions (merchant appears >= n months with similar amounts)
months_seen = 6

monthly_counts = df_cleaned.groupby(['Description', 'Month'])['Amount'].sum().reset_index()
recurrence = monthly_counts.groupby('Description').size().reset_index(name='months_seen')
recurring_merchants = recurrence[recurrence['months_seen'] >= months_seen]['Description'].tolist()
df_cleaned['is_recurring_candidate'] = df_cleaned['Description'].isin(recurring_merchants)
df_cleaned[df_cleaned['is_recurring_candidate'] == True]['Description'].value_counts()

# Groceries (costco), gas for car (costco), Insurance, Golf, DTE Electric, Spotify, Chipotle

C:\Users\matt\AppData\Local\Temp\ipykernel_7508\3052014199.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['is_recurring_candidate'] = df_cleaned['Description'].isin(recurring_merchants)


Description
COSTCO WHSE                  36
COSTCO GAS                   34
MEIJER STORE                 26
AAA INSURANCE GW EFT         20
FSP*ROYAL OAK GOLF CEN       20
FSP*ROYAL OAK GOLF CENTER    18
UBER   *TRIP                 16
SQ *OASIS GOLF PRACTIC       12
DTE ENERGY                   10
Spotify USA                  10
KROGER                       10
VISIBLE                       9
CHIPOTLE 2420                 9
CHIPOTLE 2840                 7
Name: count, dtype: int64

In [53]:
# 2) Frequent small purchases (many transactions with small avg amount)
freq = 5
mean_dollars = 25
merchant_stats = df_cleaned.groupby('Description')['Amount'].agg(['count', 'mean']).reset_index()
freq_small = merchant_stats[(merchant_stats['count'] >= freq) & (merchant_stats['mean'] < mean_dollars)]
df_cleaned['is_freq_small'] = df_cleaned['Description'].isin(freq_small['Description'])
df_cleaned[df_cleaned['is_freq_small'] == True]['Description'].value_counts()

# Driving Range, NYCT (I think this is NY subway), Spotify, Chipotle, Car Wash

C:\Users\matt\AppData\Local\Temp\ipykernel_7508\1846955959.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['is_freq_small'] = df_cleaned['Description'].isin(freq_small['Description'])


Description
FSP*ROYAL OAK GOLF CEN       20
FSP*ROYAL OAK GOLF CENTER    18
MTA*NYCT PAYGO               11
Spotify USA                  10
CHIPOTLE 2420                 9
CHIPOTLE 2840                 7
GLF*GOLFNOWRESERVATIO         7
FARMINGTON HILLS GOLF         5
VORTEX BRIGTHON               5
Name: count, dtype: int64

In [ ]:
# 3) One-off large purchases relative to merchant history (z-score)
# 3) One-off large purchases relative to overall spending behavior (global z-score)
global_mean = df_cleaned['Amount'].mean()
global_std = df_cleaned['Amount'].std()

df_cleaned['z_score_global'] = (df_cleaned['Amount'] - global_mean) / global_std
df_cleaned['is_one_off_large'] = df_cleaned['z_score_global'] > 3
df_cleaned[df_cleaned['is_one_off_large'] == True]['Description'].value_counts()

# Vegas hotel, new glasses, football tickets for 6 people, golfing in alaska for 6, whitewater rafting alaska for 6
# Flight to alaska, another flight to somewhere, passport (Was paid back for)

C:\Users\matt\AppData\Local\Temp\ipykernel_7508\634457904.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['z_score_global'] = (df_cleaned['Amount'] - global_mean) / global_std
C:\Users\matt\AppData\Local\Temp\ipykernel_7508\634457904.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['is_one_off_large'] = df_cleaned['z_score_global'] > 3


Description
MGM GRAND - FRONT DESK       1
WARBY PARKER                 1
SEATGEEK TICKETS             1
FSP*ANCHORAGE GOLF COURSE    1
SQ *NOVA RIVER RUNNERS IN    1
DELTA AIR   0062328538791    1
SQ *INTERNATIONAL EXPRESS    1
DELTA AIR   0062314001751    1
Name: count, dtype: int64